In [1]:
%cd /om2/user/valmiki/bioplnn/

/rdma/vast-rdma/user/valmiki/bioplnn


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ConvRNN(nn.Module):
    def __init__(self, input_size, kernel_size):
        super(ConvRNN, self).__init__()
        self.conv = nn.Conv2d(input_size, input_size, kernel_size, padding=1)
        self.conv.register_full_backward_hook(lambda _, __, ___: print(1))

    def forward(self, x):
        for _ in range(10):
            x = F.relu(self.conv(x))
        return x

# Example usage
input_size = 3
kernel_size = 3
height = 32
width = 32

model = ConvRNN(input_size, kernel_size)
input_data = torch.randn(16, input_size, height, width)
output = model(input_data)
loss = output.sum()
loss.backward()


1
1
1
1
1
1
1
1
1
1


In [ ]:
x = torch.randn(16, 3, 32, 32, requires_grad=True)
conv = torch.nn.Conv2d(3, 3, 3, padding=1)
conv.weight.register_hook(lambda grad: print(1))
for _ in range(10):
    x = F.relu(conv(x))
loss = x.sum()
loss.backward()

In [3]:
import numpy as np
import glob
import scipy
import os
import torch

def build_graph(path):
    connection = {}
    coordinates = []
    Nunits = []
    Adj = []
    weight_path = glob.glob(os.path.join(path, '*.npz'))[0]
    sparse_connection = scipy.sparse.load_npz(weight_path)
    mask_path = glob.glob(os.path.join(path, '*.npy'))
    if len(mask_path) == 1:
        masks = np.load(mask_path[0])
        layers = [np.stack(mask.nonzero()) for mask in masks[:-1]]
        Ny = masks.shape[2]
    else:
        layers = ['V1_indices', 'V2_indices', 'V3_indices', 'V4_indices']
        layers = [np.load(os.path.join(path, f'{layer}.npy')) for layer in layers]
        Ny = 300
    
    for coor in layers:
        coordinates.append(coor[0] * Ny + coor[1])

    for i, incoming in enumerate(coordinates[:-1]):
        adj = torch.from_numpy(sparse_connection[coordinates[i+1]][:, incoming].toarray()).float()
        Adj.append(adj)

    return Adj

In [4]:
path = "connection"
weight_path = glob.glob(os.path.join(path, '*.npz'))[0]
sparse_connection = scipy.sparse.load_npz(weight_path)

In [5]:
layers = ['V1_indices', 'V2_indices', 'V3_indices', 'V4_indices']
layers = [np.load(os.path.join(path, f'{layer}.npy')) for layer in layers]
Ny = 300

coordinates = []
for coor in layers:
    coordinates.append(coor[0] * Ny + coor[1])

In [6]:
adj = build_graph(path)

NameError: name 'build_graph' is not defined

In [26]:
sparse_connection = scipy.sparse.load_npz(weight_path).tocoo()
values = torch.randn_like(torch.tensor(sparse_connection.data))
indices = torch.tensor(np.vstack((sparse_connection.row, sparse_connection.col)))
weight = torch.sparse_coo_tensor(indices, values, sparse_connection.shape)

In [28]:
type(weight)

torch.Tensor

In [29]:
torch.save(weight, "connection/sparse.pt")

In [30]:
torch.load('weight.pt')

tensor(indices=tensor([[   35,    35,    35,  ..., 33479, 33479, 33750],
                       [   64,    68,    69,  ..., 28677, 28976, 29555]]),
       values=tensor([-0.4614,  0.0074, -1.2774,  ..., -0.1914, -1.1798,
                      -0.5228]),
       size=(45000, 45000), nnz=169035, dtype=torch.float64,
       layout=torch.sparse_coo)

In [24]:
adj[2].shape

torch.Size([4299, 4244])